# Using the Amazon Rekognition API 

In this notebook, you will use object and celebrity detection, content moderation, text detection, face detection, and face collections Amazon Rekognition API.

***
This notebook provides a walkthrough of the different [ API's]() in Amazon Rekognition to identify objects, celebrities, faces, content, and text.  This notebook utilizes the Python [ Boto3 SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html).
***

# Initialize SageMaker Notebook Environment

In [ ]:
# Initialise Notebook
# Import the AWS boto3 Python SDK
import time
import os
import boto3
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ImageFont


In [ ]:
# Create a boto3 session and get the current AWS Region this notebook is using.

mySession = boto3.session.Session()
awsRegion = mySession.region_name

In [ ]:
# Initialize the S3 and Rekognition clients.

rekognition = boto3.client('rekognition')
s3 = boto3.client('s3')

In [ ]:
# Set the S3 bucket that contains sample images and videos

# We are provide sample images and videos in this bucket, so
# you do not have to download/upload test images and videos.

bucketName = "m2c-gps304-workshop-" + awsRegion

In [ ]:
# Create a temporary directory
# This directory is not needed to call Rekognition APIs.
# We will only use this directory to download images from the S3 bucket and draw bounding boxes

!mkdir temp
tempFolder = 'temp/'

# Detect labels in image

A label or a tag is an object, scene, action, or concept found in an image or video based on its contents. For example, a photo of people on a tropical beach may contain labels such as Palm Tree (object), Beach (scene), Running (action), and Outdoors (concept). 
***

### Set the image to you will submit to Amazon Rekognition.

In [ ]:
imageLocation = "media/object-detection/cars.png"

### View the image that you will be submitting to the detect images Rekognition API.

In [ ]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': imageLocation})))

#### Call Rekognition to detect objects in the image

In [ ]:
# Call Amazon Rekognition to detect objects in the image
# https://docs.aws.amazon.com/rekognition/latest/dg/API_DetectLabels.html

detectLabelsResponse = rekognition.detect_labels(
    Image={
        'S3Object': {
            'Bucket': bucketName,
            'Name': imageLocation,
        }
    }
)

#### Review the raw JSON reponse from Rekognition

In [ ]:
# Show JSON response returned by Rekognition Labels API (Object Detection).
# In the JSON response below, you will see Label, detected instances, confidence score, and additional information.

display(detectLabelsResponse)

### Display a list of labels detected

In [ ]:
for label in detectLabelsResponse["Labels"]:
    print("- {} (Confidence: {})".format(label["Name"], label["Confidence"]))

# Recognize celebrities in image
***

### Set the image to you will submit to Amazon Rekognition.

In [ ]:
imageLocation = "media/celebrity-recognition/GrandTourjc.png"

### View the image that you will be submitting to the detect images Rekognition API.

In [ ]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': imageLocation})))

#### Call Rekognition to recognize celebrities in the image

In [ ]:
# Call Amazon Rekognition to recognize celebrities in the image.
# https://docs.aws.amazon.com/rekognition/latest/dg/API_RecognizeCelebrities.html

recognizeCelebritiesResponse = rekognition.recognize_celebrities(
    Image={
        'S3Object': {
            'Bucket': bucketName,
            'Name': imageLocation,
        }
    }
)

#### Review raw JSON reponse from Rekognition

In [ ]:
# Show JSON response returned by Rekognition Celebrity Recognition API
# In the JSON response below, you will see CelebrityFaces which contains information about recognized celebrities.
# For each recognized celebrity, you will see information like Name, Id, Urls, and additional information about 
# their facial attributes.

display(recognizeCelebritiesResponse)

### Review celebreties detected

In [ ]:
for celibrity in recognizeCelebritiesResponse["CelebrityFaces"]:
    print("- {} (Confidence: {})".format(celibrity["Name"], celibrity["MatchConfidence"]))

#### Show image with bounding boxes around recognized celebrities

In [ ]:
# Define a function that will display an image with bounding boxes around recognized celebrities
# We will call this function in the next step
  
def drawBoundingBoxes (sourceImage, boxes):
    # blue, green, red, grey
    colors = ((255,255,255),(255,255,255),(76,182,252),(52,194,123))
    
    # Download image locally
    imageLocation = tempFolder+os.path.basename(sourceImage)
    s3.download_file(bucketName, sourceImage, imageLocation)

    # Draws BB on Image
    bbImage = Image.open(imageLocation)
    draw = ImageDraw.Draw(bbImage)
    width, height = bbImage.size
    col = 0
    maxcol = len(colors)
    line= 3
    for box in boxes:
        x1 = int(box[1]['Left'] * width)
        y1 = int(box[1]['Top'] * height)
        x2 = int(box[1]['Left'] * width + box[1]['Width'] * width)
        y2 = int(box[1]['Top'] * height + box[1]['Height']  * height)
        
        draw.text((x1,y1),box[0],colors[col])
        for l in range(line):
            draw.rectangle((x1-l,y1-l,x2+l,y2+l),outline=colors[col])
        col = (col+1)%maxcol
    
    imageFormat = "PNG"
    ext = sourceImage.lower()
    if(ext.endswith('jpg') or ext.endswith('jpeg')):
        imageFormat = 'JPEG'

    bbImage.save(imageLocation,format=imageFormat)

    display(bbImage)

In [ ]:
# Extract bounding box information from the JSON response above and display an image with bounding boxes around celebrities.

boxes = []

celebrities = recognizeCelebritiesResponse['CelebrityFaces']

for celebrity in celebrities:
    boxes.append ((celebrity['Name'], celebrity['Face']['BoundingBox']))
    
drawBoundingBoxes(imageName, boxes)

# Content moderation in Images
***

### Set the image to you will submit to Amazon Rekognition.

In [ ]:
imageLocation = "media/content-moderation/yoga_swimwear_resized.jpg"

### View the image that you will be submitting to the detect images Rekognition API.

In [ ]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': imageLocation})))

#### Call Rekognition to detect objects in the image

In [ ]:
# Call Amazon Rekognition to detect unsafe content in the image
# https://docs.aws.amazon.com/rekognition/latest/dg/moderation.html

detectModerationLabelsResponse = rekognition.detect_moderation_labels(
   Image={
       'S3Object': {
           'Bucket': bucketName,
           'Name': imageLocation,
       }
   }
)

#### Review the raw JSON reponse from Rekognition

In [ ]:
# Show JSON response returned by Rekognition Moderation API
# In the JSON response below, you will see Moderation Labels, confidence score and additional information.

display(detectModerationLabelsResponse)

#### Display a list of detected moderation labels

In [ ]:
for label in detectModerationLabelsResponse["ModerationLabels"]:
    print("- {} (Confidence: {})".format(label["Name"], label["Confidence"]))
    print("  - Parent: {}".format(label["ParentName"]))

# Text Detection
***

### Set the image you will submit to Amazon Rekognition.

In [ ]:
imageLocation = "media/text-detection/blender_tearsofsteel.jpg"

### View the image submitting to Rekognition

In [ ]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': imageLocation})))

### Call Rekognition to detect text

In [ ]:
# Call Amazon Rekognition to detect text in the image
# https://docs.aws.amazon.com/rekognition/latest/dg/text-detection.html

detectTextResponse = rekognition.detect_text(
    Image={
        'S3Object': {
            'Bucket': bucketName,
            'Name': imageLocation,
        }
    },
)

#### Review the raw JSON reponse from Rekognition

In [ ]:
display(detectTextResponse)

#### Display list of detected text

In [ ]:
for text in detectTextResponse["TextDetections"]:
    print("- {} (Confidence: {})".format(text["DetectedText"], text["Confidence"]))

# Face Detection
***

### Set the image to you will submit to Amazon Rekognition.

In [ ]:
imageLocation = "media/face-detection/face-detection.jpg"

### View the image that's being submitted

In [ ]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': imageLocation})))

### Submit face detection

In [ ]:
detectFacesResponse = rekognition.detect_faces(
    Image={
        'S3Object': {
            'Bucket': bucketName,
            'Name': imageLocation,
        }
    },
    Attributes=[
        'ALL',
    ]
)

#### Review the raw JSON reponse from Rekognition

In [ ]:
display(detectFacesResponse)

# Face Collections
***

### Set the name of your collection id

In [ ]:
collectionName = "M2C-GPS304-Face-Collection"

### Create face collection  

In [ ]:
createCollectionResponse=rekognition.create_collection(CollectionId=collectionName)

### View the results of your face collection

In [ ]:
display(createCollectionResponse)

### Set the image to you will submit to the face collection.

In [ ]:
imageLocation = "media/face-collection/face1a.jpeg"

### View the image that's being submitted to the face collection

In [ ]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': imageLocation})))

### Crate a face in the collection

In [ ]:
# Set the image name used to submit to the face collection
fileName = os.path.basename(imageName)

faceIndexResponse=rekognition.index_faces(
                    CollectionId=collectionName,
                    Image={
                        'S3Object':{
                            'Bucket':bucketName,
                            'Name':imageLocation
                            }
                        },
                    ExternalImageId=os.path.splitext(imageName)[0],
                    MaxFaces=1,
                    QualityFilter="AUTO",
                    DetectionAttributes=['ALL']
                    )

### View the response for indexing the face

In [ ]:
display(faceIndexResponse)

#### Set face images to upload to the face collection

In [ ]:
faceLocations = [
    'media/face-collection/face2.jpg',
    'media/face-collection/face3.jpg',
    'media/face-collection/face4.jpg',
    'media/face-collection/face5.jpg',
    'media/face-collection/face6.jpg'
    ]

### Upload faces to face collection

In [ ]:
for face in faceLocations:
    fileName = os.path.basename(face)
    faceIndexResponse=rekognition.index_faces(
                    CollectionId=collectionName,
                    Image={
                        'S3Object':{
                            'Bucket':bucketName,
                            'Name':face
                            }
                        },
                    ExternalImageId=os.path.splitext(fileName)[0],
                    MaxFaces=1,
                    QualityFilter="AUTO",
                    DetectionAttributes=['ALL']
                    )

#### Review the faces in the face collection

In [ ]:
listFacesResponse = rekognition.list_faces(
    CollectionId=collectionName,
)

#### Review the raw JSON reponse from Rekognition

In [ ]:
display(listFacesResponse)

### Set the location of the image used to search faces for a match

In [ ]:
imageLocation = "media/face-collection/face1b.jpeg"

### View image

In [ ]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': imageLocation})))

### Search for similar faces in face collection

In [ ]:
searchFacesByImageResponse = rekognition.search_faces_by_image(
    CollectionId=collectionName,
    Image={
        'S3Object': {
            'Bucket': bucketName,
            'Name': imageName,
        }
    },
)

#### Review the raw JSON reponse from Rekognition

In [ ]:
display(searchFacesByImageResponse)